In [15]:
E = 10800 #N/mm2
I = 14.06 * (10**6)
Width = 0.86
Length = 2900
sigma = (5 * Width * Length**4) / (384 * E * I)
print(sigma, "mm")

DesignStillwaterDepth = 3



5.215790591852818 mm


In [28]:
################################### GET COLUMN PROPERTIES ########################################
# Loaded info
material = "steel"
Length = 3 # m
Height = 0.6 # m
Width = 0.2 # m
enviroment = "city"

########################## GET WATER TYPE AND FLOOD DEPTH + FLOOD RISK #########################
WaterType = "salt" # majority of seas and rivers in the netherlands are salty
DesignStillwaterDepth = 3 # m

################################### GET VALUES FROM GIVEN VARIABLES ################################
#  Modulus of elasticity in MPa
match material:
    # Source: https://www.calculand.com/unit-converter/stoffe-liste.php?gruppe=Elastic+modulus+%28E%29%2C+Young%27s+modulus&einheit=1e6--MPa
    case "concrete":
        Elasticity =  15000 # MPA concrete
    case "wood":
        Elasticity =  7000 # MPA wood 
    case "steel":
        Elasticity =  210000 # MPA steel

# Second moment of area in m4
match material:
    case "concrete":
        Width = Width * 1000
        Height = Height * 1000
        Inertia = ((Width * (Height**3))) / 12 #
    case "wood":
        Width = Width * 1000
        Height = Height * 1000
        Inertia = ((Width * (Height**3)) / 12) #  
    case "steel":
        Width = Width * 1000
        Height = Height * 1000
        Inertia = 7763 * 1000 # cm^4 to m4 steel HE 260A

# specific weights per water type
match WaterType:
    case "salt":
        SpecificWeightWater = 64.0 * 16.0185 # lb/ft3 to kg/m3
    case "fresh":
        SpecificWeightWater = 62.4 * 16.0185 # lb/ft3 to kg/m3

# Debris weight
match enviroment:
    case "city":
        DebrisWeight =  1500+454 # car weight and debris suggestion
    case "village":
        DebrisWeight =  655 # tree weight https://www.researchgate.net/figure/Summary-of-tree-heights-weights-and-volumes_tbl1_232354725
    case "plains":
        DebrisWeight =  77 # wood weight

# t = duration of impact in seconds
match material:
# City of Honolulu building code for impact durations
    case "concrete":
        impacttime = 0.1 # second
    case "wood":
        impacttime = 1 # second 
    case "steel":
        impacttime = 0.5 # second

# Max deflection limits Column
match material:
     # Source: Eurocode 5, 3, 2 (in order)
    case "concrete":
        PermittedDeflection =  (Length * 1000) / 500 # EUC 5 Concrete
    case "wood":
        PermittedDeflection =  (Length * 1000) / 300 # EUC 3 wood 
    case "steel":
        PermittedDeflection =  (Length * 1000) / 500 # EUC 2 steel
        
        GravitationalConstant = 9.81 # m/s
Velocity = (GravitationalConstant * DesignStillwaterDepth)**0.5 # V = (gds)0.5

# Breaking wave calculation
def breakingwave():
    DragCoefficient = 2.25 # if square
    HeightMetric = Height * 0.3048
    BreakingWaveOnPiles = 0.5 * DragCoefficient * SpecificWeightWater * HeightMetric * (DesignStillwaterDepth**2)
    return BreakingWaveOnPiles

# Hydrostatic
def hydrostatic():
    HydrostaticLoad = 0.5 * SpecificWeightWater * (DesignStillwaterDepth**2) # kg
    return HydrostaticLoad

# Couldn't convert to metric, thus this function will be in imperial
# Hydrodynamic (Fluid dynamics)
def hydrodynamic():
    CrossArea = Height * Width
    DragCoefficient = 2 # square
    match WaterType:
        case "salt":
            MassDensityFluid = 1.99  # slugs/ft3 
        case "fresh":
            MassDensityFluid = 1.94  # slugs/ft3 
    VelocityImp = Velocity * 3.2808399
    CrossAreaImp = CrossArea * 10.7639
    HydrodynamicLoad = 0.5 * DragCoefficient * MassDensityFluid * (VelocityImp**2) * CrossAreaImp
    return HydrodynamicLoad

def debrisimpact():
    DebrisLoad = (DebrisWeight * Velocity) / (GravitationalConstant * impacttime)
    return DebrisLoad

# Calculate all necessary loads
BreakingWaves = breakingwave()
HydrodynamicLoad = hydrodynamic()
HydrostaticLoad = hydrostatic()
DebrisLoad = debrisimpact()

# convert weight into forces
LoadBreak = (BreakingWaves * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 
LoadCombination = ((HydrodynamicLoad + HydrostaticLoad) * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 
LoadDebris = (DebrisLoad * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 

# Calculate reaction loads
def reactionloads(Load):
    ReactionloadR1 = ((Load * (DesignStillwaterDepth * 1/3)**2) / (2* Length**3)) * ((DesignStillwaterDepth*2/3) + (2*Length))
    ReactionloadR2 = ((Load * (DesignStillwaterDepth * 2/3)) / (2* (Length**3))) * ((3 * (Length**2)) - ((DesignStillwaterDepth*2/3)**2))
    return ReactionloadR1, ReactionloadR2

# Calculate moments for breaking loads and load combinations
def moments(Load, ReactionLoads):
    # formulas from structx
    MomentLoad = ReactionLoads[0] * (DesignStillwaterDepth * 2/3)
    MomentFixed = ((Load * (DesignStillwaterDepth * 2/3) * (DesignStillwaterDepth * 1/3)) / (2 * (Length**2))) * ((DesignStillwaterDepth * 2/3) + Length)
    return MomentLoad, MomentFixed

# calculate max deflection for both breaking loads and the load combination
def deflection(Load):
    Load = Load * 1000
    DesignStillwaterDepthmm = DesignStillwaterDepth * 1000
    Lengthmm = Length * 1000
    Widthmm = Width * 1000
    # formulas from structx
    if (DesignStillwaterDepthmm * 2/3) < (0.414 * Length):
        MaxDeflection = (Load * (DesignStillwaterDepthmm * 2/3) / (3 * Elasticity * Inertia)) * (((Lengthmm**2) - ((DesignStillwaterDepthmm * 2/3)**2))**3) / ((3 * (Lengthmm**2) - ((DesignStillwaterDepthmm * 2/3)**2))**2)
    elif (DesignStillwaterDepthmm * 2/3) > (0.414 * Length):
        MaxDeflection = ((Load * (DesignStillwaterDepthmm * 2/3) * ((DesignStillwaterDepthmm * 1/3)**2)) / (6 * Elasticity * Inertia)) * math.sqrt((DesignStillwaterDepthmm * 2/3) / ((2*Lengthmm)+(DesignStillwaterDepthmm * 2/3)))
    return MaxDeflection


MaxdefBreak = deflection(LoadBreak) 
MaxdefCombination = deflection(LoadCombination) 
MaxdefDebris = deflection(LoadDebris) 

# check if max deflection exceeds permitted deflection
def delfectioncheck(Maxdef):
    if Maxdef < PermittedDeflection:
        print("Deflection: Risk detected", "permitted amount is", round(PermittedDeflection, 3), "mm occuring amount is", round(Maxdef * 1000, 3), "mm")
    else:
        print("Deflection: Column is OK", "permitted amount is", round(PermittedDeflection, 3), "mm occuring amount is", round(Maxdef * 1000, 3), "mm")

print("Deflection check for Breaking Loads...")
delfectioncheck(MaxdefBreak)

Deflection check for Breaking Loads...
Deflection: Column is OK permitted amount is 6.0 mm occuring amount is 1903846.547 mm
